In Approach one, we will process the data using a ratio rather than the usual normalization. For example we will take all the cashflow values as a ratio to net income, and all the balance sheet values as a ratio to current assets. We can compare the performance between the two.

In [16]:
import os
import pandas as pd
import dateutil.relativedelta
from datetime import datetime

In [19]:



parent_dir = os.path.join(os.getcwd() , "Data/Raw")
OLS_input = {}
#basic_info_check = ["ebitdaMargins" , "profitMargins"  , "grossMargins", "debtToEquity", "returnOnEquity" ]
cashflow_info_check = ["Change To Liabilities", "Total Cash From Operating Activities", "Net Borrowings" , "Total Cashflows From Investing Activities" , "Investments", "Net Income"]
balance_sheet_info_check = ["Total Liab" , "Total Assets", "Retained Earnings", "Cash", "Net Receivables", "Long Term Debt", "Short Long Term Debt", "Total Current Assets"]

for file in os.listdir(parent_dir):
    path = os.path.join(parent_dir , file)
        

    ### Get variables from Cash Flow Statement ###
    cashflow_info = pd.read_csv(path + "/cashflow.csv")
    balance_sheet = pd.read_csv(path + "/stock_balance_sheet.csv")
    
    if(cashflow_info.empty or balance_sheet.empty):
        continue
 

    if(len(cashflow_info.columns) != len(balance_sheet.columns)):
        print("EEROROR")
    
    if("." in file):
        continue
    

    for info_item in cashflow_info_check:
        
        if(info_item in cashflow_info["Unnamed: 0"].values):

            data =  cashflow_info[cashflow_info["Unnamed: 0"] == info_item].values[0][1::]
            data = data.tolist()
            if len(data) != 4:

                print("Cash flow OOF" , file)
            # data = transformer.transform(data.reshape(1, -1))
            # data = data.tolist()

        else:
            data = [None] * (len(balance_sheet.columns)-1)

        if info_item in OLS_input:

            OLS_input[info_item] += data
        else:
            OLS_input[info_item] = data

    ### Get variables from balance sheet ###



    for info_item in balance_sheet_info_check:
        if(info_item in balance_sheet["Unnamed: 0"].values):
            
            data =  balance_sheet[balance_sheet["Unnamed: 0"] == info_item].values[0][1::]
            data = data.tolist()
            if len(data) != 4:
                print("Balance SHEET OOF", file)
            # data = transformer.transform(data.reshape(1, -1))
            # data = data.tolist()    
        else:
            data = [None] * (len(balance_sheet.columns)-1)

        if info_item in OLS_input:
            OLS_input[info_item] += data
        else:
            OLS_input[info_item] = data
            
    
    ### Get the returns of the stock over the quater
    
    times = balance_sheet.columns.values[1::]
    times = list(map(datetime.fromisoformat, times))
    stock_price_history = pd.read_csv(path + "/stock_prices.csv")
    stock_price_history["Date"] = pd.to_datetime(stock_price_history["Date"])
    quarterly_returns = []


    for i in range(len(times)):
        quarterly_data = pd.DataFrame()
        quarterly_data = stock_price_history[(stock_price_history["Date"] < times[i] ) & (stock_price_history["Date"] > times[i] - dateutil.relativedelta.relativedelta(months=3))].reset_index(drop=True)
        #calculate the net return in each quater
        if not(quarterly_data.empty) :
            total_ret = (quarterly_data.loc[len(quarterly_data)-1]["Close"] - quarterly_data.loc[0]["Open"])/quarterly_data.loc[0]["Open"]
            quarterly_returns.append(total_ret)
        else:
            quarterly_returns.append(None)
        
    if len(quarterly_returns) != 4:
        print("RETURN OOF", file)

    if "Returns" not in OLS_input:
        OLS_input["Returns"] = quarterly_returns
    else:
        OLS_input["Returns"] += quarterly_returns

  
for key , val in OLS_input.items():
    print(key , len(val))

df = pd.DataFrame(OLS_input)
df.to_csv(os.path.join(os.getcwd() , "Data") + "/Processed/linear_reg_data.csv")

Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
RETURN OOF 259960.KS
Cash flow OOF IOT
Cash flow OOF IOT
Cash flow OOF IOT
Cash flow OOF IOT
Cash flow OOF IOT
Balance SHEET OOF IOT
Balance SHEET OOF IOT
Balance SHEET OOF IOT
Balance SHEET OOF IOT
Balance SHEET OOF IOT
Balance SHEET OOF IOT
RETURN OOF IOT
Cash flow OOF OVH.PA
Cash flow OOF OVH.PA
Cash flow OOF OVH.PA
Cash flow OOF OVH.PA
Cash flow OOF OVH.PA
Balance SHEET OOF OVH.PA
Balance SHEET OOF OVH.PA
Balance SHEET OOF OVH.PA
Balance SHEET OOF OVH.PA
Balance SHEET OOF OVH.PA
Balance SHEET OOF OVH.PA
Balance SHEET OOF OVH.PA
RETURN OOF OVH.PA
Cash flow OOF SST
Cash flow OOF SST
Cash flow OOF SST
Cash flow OOF SST
Cash flow OOF SST
Balance SHEET OOF SST
Balance SHEET OOF SS

In [14]:
df = pd.read_csv("Data/Processed/linear_reg_data.csv")
df = df.dropna().drop(columns=["Unnamed: 0"]).reset_index(drop=True)
df

,Change To Liabilities,Total Cash From Operating Activities,Net Borrowings,Total Cashflows From Investing Activities,Investments,Net Income,Total Liab,Total Assets,Retained Earnings,Cash,Net Receivables,Long Term Debt,Short Long Term Debt,Total Current Assets,Returns
0,-2.067230e+11,6.266689e+12,4.503500e+10,-6.350965e+12,-3.286883e+12,3.310823e+12,2.455460e+13,8.326905e+13,5.246040e+13,2.761977e+12,7.545234e+12,1.176780e+13,2.311561e+12,2.491050e+13,-0.215686
1,-1.101190e+11,4.549973e+12,1.655130e+11,-3.484309e+12,-2.414150e+11,1.984514e+12,2.283988e+13,7.771790e+13,4.915642e+13,2.666351e+12,6.522701e+12,1.153141e+13,2.167540e+12,2.009956e+13,-0.070632
2,1.891400e+10,3.206762e+12,1.900472e+12,-5.885210e+12,-1.747571e+12,9.904470e+11,2.273942e+13,7.511801e+13,4.717885e+13,2.226144e+12,5.379719e+12,1.078574e+13,2.658938e+12,1.838488e+13,0.080321
3,3.116740e+11,3.261827e+12,-5.049000e+10,-2.979418e+12,-4.332600e+10,1.766796e+12,1.926476e+13,7.117385e+13,4.699573e+13,2.975989e+12,5.241949e+12,8.199087e+12,2.934671e+12,1.657095e+13,0.427711
4,1.027017e+12,2.063545e+13,-1.333413e+12,8.722370e+11,1.575785e+13,1.064379e+13,1.217212e+14,4.266212e+14,2.930648e+14,3.903142e+13,4.521067e+13,5.097320e+11,5.180650e+11,2.181632e+14,0.059540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1.400000e+06,4.650000e+07,4.968000e+08,-6.604000e+08,3.010000e+07,-4.060000e+07,4.773400e+09,6.608100e+09,-3.210000e+07,1.968000e+08,1.212000e+08,1.232400e+09,5.300000e+06,4.087000e+08,0.178077
496,2.100000e+07,8.860000e+07,4.968000e+08,2.000000e+07,5.790000e+07,9.300000e+06,1.831600e+09,2.823300e+09,8.500000e+06,3.329000e+08,1.145000e+08,7.395000e+08,6.500000e+06,5.300000e+08,0.079830
497,4.440000e+07,9.300000e+07,-6.400000e+06,-9.900000e+07,-9.410000e+07,3.200000e+06,1.600100e+09,2.547000e+09,-8.000000e+05,2.291000e+08,1.184000e+08,7.392000e+08,7.800000e+06,4.855000e+08,0.020408
498,-2.451000e+06,1.025100e+07,-1.111000e+06,-5.616000e+06,-3.995000e+06,-1.766100e+07,2.426200e+08,4.141270e+08,-4.816830e+08,1.011160e+08,6.131200e+07,5.740000e+05,4.397000e+06,2.866320e+08,0.114943


Now we proccess the data in a predictive manner, we want to see if these factors can predict the returns of a 2 week holding period after the quarterly reports come out

In [25]:

parent_dir = os.path.join(os.getcwd() , "Data/Raw")
OLS_input = {}
#basic_info_check = ["ebitdaMargins" , "profitMargins"  , "grossMargins", "debtToEquity", "returnOnEquity" ]
cashflow_info_check = ["Change To Liabilities", "Net Borrowings" , "Investments", "Net Income"]
balance_sheet_info_check = ["Total Liab" , "Total Assets", "Retained Earnings", "Cash", "Net Receivables", "Total Current Liabilities", "Total Current Assets"]

for file in os.listdir(parent_dir):
    path = os.path.join(parent_dir , file)
        

    ### Get variables from Cash Flow Statement ###
    cashflow_info = pd.read_csv(path + "/cashflow.csv")
    balance_sheet = pd.read_csv(path + "/stock_balance_sheet.csv")
    
    if(cashflow_info.empty or balance_sheet.empty):
        continue
 

    if(len(cashflow_info.columns) != len(balance_sheet.columns)):
        print("EEROROR")

    if("." in file):
        continue
    

    for info_item in cashflow_info_check:
        
        if(info_item in cashflow_info["Unnamed: 0"].values):

            data =  cashflow_info[cashflow_info["Unnamed: 0"] == info_item].values[0][1::]
            data = data.tolist()

        else:
            data = [None] * (len(balance_sheet.columns)-1)

        if info_item in OLS_input:

            OLS_input[info_item] += data
        else:
            OLS_input[info_item] = data

    ### Get variables from balance sheet ###



    for info_item in balance_sheet_info_check:
        if(info_item in balance_sheet["Unnamed: 0"].values):
            
            data =  balance_sheet[balance_sheet["Unnamed: 0"] == info_item].values[0][1::]
            data = data.tolist()
            # data = transformer.transform(data.reshape(1, -1))
            # data = data.tolist()    
        else:
            print(info_item, file)
            data = [None] * (len(balance_sheet.columns)-1)

        if info_item in OLS_input:
            OLS_input[info_item] += data
        else:
            OLS_input[info_item] = data
            
    
    ### Get the returns of the stock over the quater
    
    times = balance_sheet.columns.values[1::]
    times = list(map(datetime.fromisoformat, times))
    stock_price_history = pd.read_csv(path + "/stock_prices.csv")
    stock_price_history["Date"] = pd.to_datetime(stock_price_history["Date"])
    quarterly_returns = []


    for i in range(len(times)):
        quarterly_data = pd.DataFrame()
        quarterly_data = stock_price_history[(stock_price_history["Date"] > times[i] ) & (stock_price_history["Date"] < times[i] + dateutil.relativedelta.relativedelta(weeks=2))].reset_index(drop=True)

        #calculate the net return in each quater
        if not(quarterly_data.empty) :
            total_ret = (quarterly_data.loc[len(quarterly_data)-1]["Close"] - quarterly_data.loc[0]["Open"])/quarterly_data.loc[0]["Open"]
            quarterly_returns.append(total_ret)
        else:
            quarterly_returns.append(None)
        

    if "Returns" not in OLS_input:
        OLS_input["Returns"] = quarterly_returns
    else:
        OLS_input["Returns"] += quarterly_returns

  
for key , val in OLS_input.items():
    print(key , len(val))

df = pd.DataFrame(OLS_input)
df.to_csv(os.path.join(os.getcwd() , "Data") + "/Processed/predict_linear_reg_data.csv")


Net Receivables CYN
Net Receivables DAVE
Retained Earnings EQIX
Retained Earnings ERIC
Net Receivables MVIS
Net Receivables NHTC
Net Receivables POSH
Net Receivables RVSN
Retained Earnings SIMO
Retained Earnings SST
Net Receivables TCDA
Net Receivables WULF
Change To Liabilities 2324
Net Borrowings 2324
Investments 2324
Net Income 2324
Total Liab 2324
Total Assets 2324
Retained Earnings 2324
Cash 2324
Net Receivables 2324
Total Current Liabilities 2324
Total Current Assets 2324
Returns 2324
